In [16]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import zero_one_loss

In [17]:
# Setting up data input types. 

#Train 

X_train = np.loadtxt('/home/aditya/Documents/UIUC/spring_2016/STAT427/project/uci_har_dataset/train/X_train.txt')
y_train = np.loadtxt('/home/aditya/Documents/UIUC/spring_2016/STAT427/project/uci_har_dataset/train/y_train.txt', dtype=np.int)
persons_train = np.loadtxt('/home/aditya/Documents/UIUC/spring_2016/STAT427/project/uci_har_dataset/train/subject_train.txt', dtype=np.int)

#Test 
X_test = np.loadtxt('/home/aditya/Documents/UIUC/spring_2016/STAT427/project/uci_har_dataset/test/X_test.txt')
y_test = np.loadtxt('/home/aditya/Documents/UIUC/spring_2016/STAT427/project/uci_har_dataset/test/y_test.txt', dtype=np.int)
persons_test = np.loadtxt('/home/aditya/Documents/UIUC/spring_2016/STAT427/project/uci_har_dataset/test/subject_test.txt', dtype=np.int)

# X and Y for both axises. 
X_all = np.concatenate([X_train, X_test])
y_all = np.concatenate([y_train, y_test])
    
feature_names = [x.split(' ')[1] for x in open('/home/aditya/Documents/UIUC/spring_2016/STAT427/project/uci_har_dataset/features.txt').read().split('\n') if len(x) > 0]


In [18]:
def SVM_feature_extraction():
    clf = svm.LinearSVC
    clf.fit(X_train,y_train)
    X_train_t = clf.decision_function(X_train)
    X_test_t = clf.decision_function(X_test)
    
    return (X_train_t, X_test_t)


In [19]:
def run_clfs_on_data(classifiers, Xs, ys, add_last_action = False):
    results = {}
    for name, clf in classifiers.iteritems():
        print "running %s" % name
        clf_results = fit_clf_kfold(clf['clf'], Xs, ys, flatten=not clf['structured'], add_last_action=add_last_action)
        # with feature selection:
        clf_results = fit_clf_kfold(clf['clf'], [X[:,select_features] for X in X_pers_all], y_pers_all,flatten=not clf['structured'])
        results[name] = clf_results
    return results


In [20]:
def plot_most_important_features(clf, label_names, feature_names, n=10, best=True, absolut=True):
    if absolut:
        ranked_features = np.argsort(np.abs(clf.coef_), axis=None)
    else:
        ranked_features = np.argsort(clf.coef_, axis=None)
        
    if best:
        ranked_features = ranked_features[::-1] #inverse to get the best first
        
    for i, fweights_idx in enumerate(ranked_features[:n]):
            label_idx,feature_idx = np.unravel_index(fweights_idx, clf.coef_.shape)
            print "%d. f: %s\t\t c: %s\t value: %f" % (i, feature_names[feature_idx], label_names[label_idx], clf.coef_[(label_idx,feature_idx)])


In [22]:
def main(X_train,y_train,persons_train, X_test, y_test, persons_test, X_all, y_all): 
    print "Data:"
    X_train_pers, y_train_pers = unflatten_per_person(X_train, y_train, persons_train)
    X_test_pers, y_test_pers = unflatten_per_person(X_test, y_test, persons_test)
